# Purpose

### 2022-02-14
In this notebook I'll select the clusters for the new FPR experiments for Canada, UK, Australia, & India.

Note that this is supposed to be an SFW experiment, so we'll need to filter out subreddits that are `over_18` or rated as `X`.

In one sheet include BOTH subreddit names & subreddit IDs.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [ ]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

In [ ]:
# colab auth for BigQuery
from google.colab import auth, files, drive
auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [ ]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [ ]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 8.0 MB/s 
     |████████████████████████████████| 14.2 MB 18.3 MB/s 
     |████████████████████████████████| 965 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 58.7 MB/s 
     |████████████████████████████████| 285 kB 50.6 MB/s 
     |████████████████████████████████| 13.2 MB 33.1 MB/s 
     |████████████████████████████████| 79.9 MB 103 kB/s 
     |████████████████████████████████| 133 kB 54.4 MB/s 
     |████████████████████████████████| 715 kB 51.2 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
     |████████████████████████████████| 112 kB 59.1 MB/s 
     |████████████████████████████████| 180 kB 48.4 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 146 kB 53.3 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 596 kB 57.1 MB/s 
     |████████████

## Regular Imports

In [ ]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm


# auth for google sheets
import gspread
from oauth2client.client import GoogleCredentials


gc = gspread.authorize(GoogleCredentials.get_application_default())

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

[autoreload of cloudpickle failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: cannot import name '_should_pickle_by_reference' from 'cloudpickle.cloudpickle' (/usr/local/lib/python3.7/dist-packages/cloudpickle/cloudpickle.py)
]


## Custom imports

In [ ]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.12
===
gspread		v: 4.0.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name in geo-relevance query
- change country name for google sheet name
- change country initial in google sheet
    - add google sheet KEY (after created)

# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [ ]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'United Kingdom';


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.relevance_percent_by_country_standardized = TRUE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 714 ms, sys: 55.1 ms, total: 769 ms
Wall time: 12.7 s


### Check df with geo + language information

In [ ]:
print(df_geo_and_lang.shape)

(3517, 26)


In [ ]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_32g6u,legaladviceuk,United Kingdom,True,0.759776,10.681203,0.010701,44,True
1,t5_2wkka,ukpersonalfinance,United Kingdom,True,0.836210,10.676053,0.014709,27,True
2,t5_2qkog,london,United Kingdom,True,0.639895,10.510362,0.013310,30,True
3,t5_2sumw,britishproblems,United Kingdom,True,0.686924,10.365783,0.008523,63,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [ ]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 8.52 s, sys: 450 ms, total: 8.97 s
Wall time: 24.5 s


### Check label outputs

In [ ]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [ ]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"40,714",52,0.13%,"8,844",17.85%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [ ]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(3517, 74) <- df_labels_target.shape
CPU times: user 308 ms, sys: 2.55 ms, total: 310 ms
Wall time: 315 ms


In [ ]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,"3,517","3,517",100.00%,0,0.00%
subreddit_id,object,"3,517","3,517",100.00%,0,0.00%
subreddit_name,object,"3,517","3,517",100.00%,0,0.00%
primary_topic,object,"2,997",52,1.74%,520,14.79%
rating_short,object,"3,340",3,0.09%,177,5.03%
over_18,object,746,1,0.13%,"2,771",78.79%
rating_name,object,"3,340",3,0.09%,177,5.03%
posts_for_modeling_count,int64,"3,517",713,20.27%,0,0.00%
k_0013_label,int64,"3,517",12,0.34%,0,0.00%
k_0023_label,int64,"3,517",22,0.63%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

### Loop

In [ ]:
%%time

col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

df_optimal_min_check = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
)

100%|██████████| 16/16 [00:01<00:00, 12.76it/s]

100%|██████████| 16/16 [00:01<00:00, 13.41it/s]

100%|██████████| 16/16 [00:01<00:00, 13.70it/s]

100%|██████████| 16/16 [00:01<00:00, 13.71it/s]

100%|██████████| 16/16 [00:01<00:00, 13.33it/s]

100%|██████████| 16/16 [00:01<00:00, 13.68it/s]

100%|██████████| 16/16 [00:01<00:00, 13.19it/s]

100%|██████████| 16/16 [00:01<00:00, 13.28it/s]

100%|██████████| 8/8 [00:17<00:00,  2.21s/it]


CPU times: user 17.4 s, sys: 284 ms, total: 17.7 s
Wall time: 17.8 s


### Display loop results

In [ ]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,"3,517",3,612,1,5.75,5.00,59,0006
1,"3,517",4,508,1,6.92,6.00,52,0006
2,"3,517",5,433,0,8.12,7.00,48,
3,"3,517",6,377,2,9.33,8.00,39,"0001, 0012"
4,"3,517",7,334,2,10.53,10.00,39,"0006, 0009"
5,"3,517",8,300,1,11.72,11.00,32,0006
6,"3,517",9,270,0,13.03,12.00,28,
7,"3,517",10,241,1,14.59,13.00,25,0010


## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [ ]:
n_min_subs_in_cluster_optimal = 9

n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'

# even if cluster at k < 20 is generic, keep it to avoid orphan subs
l_cols_labels = (
    [c for c in df_labels_target.columns
        if all([c != col_new_cluster_val, c.endswith('_label')])
        ]
)

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=n_min_subs_in_cluster_optimal,
    l_cols_labels_input=l_cols_labels,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

03:04:45 | INFO | "Concat'ing nested cluster labels..."
03:04:45 | INFO | "Getting topic mix at different depths..."
03:04:45 | INFO | "  Assigning base topic mix cols"
03:04:45 | INFO | "  Creating deepest base topic mix col..."
03:04:45 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:01<00:00, 11.80it/s]
03:04:47 | INFO | "Initializing values for strategy: aggregate_small_clusters"
03:04:47 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 68.25it/s]
03:04:47 | INFO | "(3517, 124) <- output shape"


In [ ]:
# [c for c in df_labels_target_dynamic_raw.columns if 'nested' in c]

In [ ]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,270,0,13.03,12.00,28,


In [ ]:
# # # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[104:109, :15],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

### Minor QA checks

In [ ]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
Mature Themes and Adult Content,306,8.7%,8.7%
Gaming,285,8.1%,16.8%
Music,276,7.8%,24.7%
Television | Podcasts and Streamers,154,4.4%,29.0%
Sports,145,4.1%,33.2%
Television,140,4.0%,37.1%
Animals and Pets,139,4.0%,41.1%
Gaming | Tabletop Games,138,3.9%,45.0%
Medical and Mental Health,133,3.8%,48.8%


In [ ]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,"2,299",65.4%,65.4%
1,"1,060",30.1%,95.5%
2,133,3.8%,99.3%
3,25,0.7%,100.0%


In [ ]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,"1,756",49.9%,49.9%
2,"1,116",31.7%,81.7%
3,443,12.6%,94.3%
4,161,4.6%,98.8%
5,37,1.1%,99.9%
6,4,0.1%,100.0%


In [ ]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default
3501,t5_2qj3r,quiz,Music | Mature Themes and Adult Content | Internet Culture and Memes,Gaming,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Hobbies | Funny/Humor | Gaming,Everyone,f,False
3502,t5_2tafq,cardsagainsthumanity,Music | Mature Themes and Adult Content | Internet Culture and Memes,Tabletop Games,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Tabletop Games | Gaming,Everyone,f,False
3503,t5_3ic2s,dartstalk,Music | Mature Themes and Adult Content | Internet Culture and Memes,Sports,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Anime | Fashion,Everyone,f,False
3505,t5_2xarh8,realcivilengineer,Music | Mature Themes and Adult Content | Internet Culture and Memes,Gaming,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Learning and Education | Funny/Humor,Everyone,-,False
3509,t5_30ttj,failrace,Music | Mature Themes and Adult Content | Internet Culture and Memes,-,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Meta/Reddit | Gaming,Everyone,-,False


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [ ]:
# # check subs around orphan sub
# # n_plus_minus_ = 5
# ix_orphan_ = (
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
#     .index
# )
# df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

In [ ]:
# # check other subs that are in the same cluster as orphan sub (at broadest level)
# l_cols_orphan_check = (
#     [
#         'subreddit_id',
#         col_new_cluster_topic_mix, 
#         # col_new_cluster_val,  # this can be really long and makes comparing harder
#         # col_subreddit_topic_mix,
#         'subreddit_name', 
#         col_new_cluster_name
#     ] +
#     l_cols_labels[:-5]
# )

# style_df_numeric(
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw['k_0013_label'] == 11]
#     [l_cols_orphan_check]
#     .iloc[3:14, :50]
#     ,
#     l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
#     rename_cols_for_display=True,

# )

In [ ]:
# label_k_to_reassign_ = 'k_0320_label'
# label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
# subreddit_id_orphan_ = 't5_2tt7r'

# mask_orphan_and_new_group = (
#     (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
#     (
#         (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
#         (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
#     )
# )

# # assign is similar to what we do in the dynamic function
# label_k_new_ = 'k_0118_label'
# label_val_new_col_ = f"{label_k_new_}_nested"
# new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
# c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_name
# ] = label_k_new_

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_val
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_prim_topic
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_topic_mix
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

# del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
# del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [ ]:
# check again, num of orphans should be lower than before
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,433,1,8.12,7.00,48,0013


In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

count    433.000000
mean       8.120092
std        3.820557
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       50.000000
Name: count, dtype: float64

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [ ]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(270, 6)


In [ ]:
df_cluster_for_humans.iloc[40:45, 1:]

,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
40,k_0085_label,Technology,17,"t5_2teb5, t5_2yfjw9, t5_3z8jcy, t5_37o7xa, t5_pvbtm, t5_2slpu, t5_2h57km, t5_2wav7, t5_1j1oxu, t5_2v64n, t5_2xzlq, t5_3gur6, t5_345qp, t5_58kotc, t5_17bti7, t5_3wolnx, t5_2s1dd","analogue, reptimeservices, nicoleonard, prideandpinion, chinatime, tagheuer, oppowatch, applewatch, impcat, wheelchairs, hermanmiller, mechanicalkeyboardsuk, scufgaming, turtlebeachsimulation, elegoomars, voxelab_proxima, printmaking"
41,k_0118_label,Technology | Home and Garden,13,"t5_32qsh, t5_v65py, t5_3422e, t5_2v6jk, t5_xp4km, t5_2mjm15, t5_2wf12, t5_12sgd5, t5_2t7k9, t5_10wsdh, t5_2rko1, t5_2slmj, t5_2tgfg","ukpreppers, europreppers, bushcraftuk, outdoorsgear, ulhikinguk, eusporehub, zerowaste, zerowastevegans, upcycling, alevelchemistry, cider, aeropress, nespresso"
42,k_0320_label,Technology | Home and Garden,11,"t5_2slqb, t5_2qrh5, t5_2ty4q, t5_2rf27, t5_2yv61, t5_3c9xq, t5_2t5kz, t5_32ofg, t5_2uqn5, t5_2s9a5, t5_2rva0","plumbing, cleaningtips, dyson, fixit, diyuk, finishing, plastering, buildingcodes, carpet, ikea, upholstery"
43,k_0320_label,Technology,13,"t5_3jaky, t5_2zpqj, t5_2trj7, t5_33zcb, t5_2a45a2, t5_2t66l, t5_2r2ne, t5_2ties, t5_3k5ka, t5_shi1c, t5_3kbc1, t5_2rh4s, t5_3fpdx","hiveheating, tado, nest, ukelectricians, govee, lavalamps, techtheatre, dashcam, tplinkkasa, smartlife, hikvision, ring, blinkcameras"
44,k_0320_label,Marketplace and Deals,12,"t5_3g3863, t5_327ds, t5_30fll, t5_2t0ke, t5_2294v6, t5_9pie1, t5_3eyzv, t5_2slj0, t5_3zqopd, t5_4himl, t5_2qjnx, t5_31k82u","discountcodesuk, promocodeshare, couponsuk, freebiesuk, laptopdealseurope, laptopdealsuk, pcpartsuk, buildapcsalesuk, amazonprimeuk, spottedbargainsuk, amazon, serialeturcestionline"


In [ ]:
# df_cluster_for_humans.tail(9)

### Check subs that have mature topics

Make a list of sub IDs to exclude for clean df

In [ ]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

39 <- clusters with 'Mature' in topic mix


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:9, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
0,0001,k_0013_label,Mature Themes and Adult Content,6,"t5_44faux, t5_2x56l, t5_5ay13a, t5_3y6zfn, t5_54n5c2, t5_2z8mlf","neighboursbabez, celebsinlingerie, fitfootyplayers, zwiftracing, mabelnsfw, ellieleachxx"
1,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,Mature Themes and Adult Content,11,"t5_2wxtt9, t5_5cv619, t5_2wu5o, t5_2t240k, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1","jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker"
2,0001-0001-0001-0001-0001-0002-0002-0002-0005-0009-0012-0014,k_1065_label,Mature Themes and Adult Content,13,"t5_wqh04, t5_to4e8, t5_2ba3ak, t5_42lxaq, t5_24jzkb, t5_2s9cd8, t5_24vux1, t5_4i3osf, t5_2hf2ft, t5_2z2x3, t5_4g3v2z, t5_4jmq4w, t5_rivb0","celeb_nylons, celebsintighttops, celebsinbusinesswear, celebmedia, strictlyhotties, emmerdalegirls, soapbabes, britishcelebritybabes, sexy_uk_girls, musicbabes, celebskart_pics, skysportnewsfemale, paddockwomen"
3,0001-0001-0002-0002-0002-0003-0003-0003,k_0118_label,Mature Themes and Adult Content,17,"t5_3lsdl, t5_zmlai, t5_41cbt5, t5_4tfbhq, t5_4t5b0r, t5_5dgyib, t5_5fzgb4, t5_5dl0so, t5_5ed0rd, t5_50zgf7, t5_4icf80, t5_51mqpn, t5_4mboaj, t5_29zu45, t5_34onga, t5_5gnmjv, t5_58f327","rosielewis, amberdavies, amberphilips, naomi_lester, hotmiddlefinger, nopixelgirls, asianbabesmuslima, tiannablatter, eviegarbe_, beckylynchbooty, miawoodhall, ambergill, p3ngali, annamaynard, cumvidandpic, hotlaurenalexis, dolanjermane2"
4,0001-0001-0002-0002-0002-0003-0003-0003-0006,k_0320_label,Mature Themes and Adult Content,10,"t5_4qepjs, t5_52oi1a, t5_59euk3, t5_31sgqc, t5_2al3kz, t5_4dgbcr, t5_2rd2l, t5_52xhgk, t5_4d1rbc, t5_51v2np","tadqeesyasin, ukhornythots, ukthotsx2, jensmith, joannachimonides, laetitiabrown, drama, sofiadeilamy, sofiaemilydeilamy, carlafrigoo2"
5,0001-0001-0002-0002-0002-0003-0003-0003-0006-0011-0014-0016-0025-0030,k_1840_label,Mature Themes and Adult Content,14,"t5_48i144, t5_2x72j, t5_4aukz3, t5_3ll0hi, t5_3ee9y6, t5_4u32j7, t5_72i29, t5_36nsk8, t5_3i2zbr, t5_35apq, t5_lg99y, t5_3hlch, t5_11rck8, t5_3jvo4","6kenza, chav, brogansmallfans, pritifit, mxelenaa, galacticawhore, carolinacutt, charlixcxsexy, davinamccall, grace_chatto, gracefituk, jorjasmith, jorjasmithpics, jadethirlwall"
6,0001-0001-0003-0003-0003-0004-0004-0004-0010,k_0320_label,Mature Themes and Adult Content | Celebrity,15,"t5_2s722, t5_2sk9j, t5_2t8y8, t5_2tvwu, t5_2rwx8, t5_31k2p, t5_2ylx0, t5_2u6ku, t5_2sdii, t5_2y16t, t5_5bxw5e, t5_4mudxz, t5_4at500, t5_30muol, t5_3dwus4","keiraknightley, karengillan, maisiewilliams, jennalouisecoleman, katebeckinsale, daisyridley, chloebennet, gilliananderson, kellybrook, georgiamayfoote, talialewiscolepics, rubyaeron, hollyrobinson, girlsinties, bowblouse"
7,0001-0001-0003-0003-0003-0004-0004-0004-0010-0018-0025-0028-0046,k_1560_label,Mature Themes and Adult Content | Celebrity,15,"t5_2ew45z, t5_q2cjh, t5_4tywry, t5_4qogng, t5_261zrr, t5_ohie3, t5_17nl3s, t5_4v254f, t5_5avkb7, t5_2yo3k, t5_2rz57, t5_2snpnh, t5_3fqet, t5_2sii9, t5_2ys6y","kadymcdermott, zaramcdermott, marybedford, mcloughlintwins, laurawoods, alicecassell, azzyland_pics, alexscott, rebeccaryanonly, hayleyorrantia, hayleywilliams, karengillanlegs, thehollywilloughby, bonniewright, laurenmayberry"
8,0001-0001-0003-0003-0003-0004-0004-0004-0010-0018-0025-0028-0046-0054-0061-0067-0089-0102-0113-0128-0137-0138,k_3927_label,Mature Themes and Adult Content | Celebrity,10,"t5_2zr9y, t5_2t9m5, t5_3dogba, t5_54xzk0, t5_2v71u, t5_2sxo1, t5_2vfo9, t5_2qx72, t5_owoic, t5_unq14","melissafumero, sarahmichellegellar, chanellemauricettee, charlottedyson, nataliealynlind, annakendrick, rachelriley, kristenbell, emmajones, emmamackey"


In [ ]:
# (
#     df_cluster_for_humans
#     [mask_mature_clusters_]
#     .iloc[-12:, :]
# )

- model did a good job identifying this drug cluster... but yeah def don't want to increase their reach   
    - `0008-0014-0025-0034-0036-0046-0048-0066`
    - `0008-0014-0025-0034-0036-0046-0048-0066-0190-0378`

- the drinking might be SFW... but prob not the 420
    - `0008-0014-0025-0034-0036-0047-0049-0067`
    - `0008-0014-0025-0034-0036-0047-0049-0067-0193`

In [ ]:
# might need to re-import in order for updates to show up
from subclu.models.reshape_clusters_v041 import _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

['0001',
 '0001-0001',
 '0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0002-0002-0003-0003-0003-0003-0003-0003',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0002-0002-0003-0003-0004',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0003-0003',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0003-0003-0003-0004-0004-0006-0007-0009-0010-0010-0010',
 '0001-0001-0001-0001-0001-0001-0001-0001-0003-0005-0006-0007-0009-0009-0012-0012-0018-0020-0023-0026-0026-0026',
 '0001-0001-0001-0001-0001-0002-0002-0002',
 '0001-0001-0002-0002-0002-0003-0003-0003',
 '0001-0001-0003-0003-0003-0004-0005-0005',
 '0002',
 '0002-0002',
 '0002-0002-0005-0005-0005',
 '0002-0002-0005-0005-0005-0007-0008-0009',
 '0004',
 '0005',
 '0005-0007-0012',
 '0008-001

### Add the flag to exclude from QA & the list of sub names to df-raw

In [ ]:
val_exclude_from_qa_ = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'

df_cluster_for_humans[col_exclude_from_qa] = np.where(
    df_cluster_for_humans[col_new_cluster_val].isin(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_),
    val_exclude_from_qa_,
    'keep',  
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")
display(value_counts_and_pcts(df_cluster_for_humans[col_exclude_from_qa]))


df_labels_target_dynamic_raw = (
    df_labels_target_dynamic_raw
    # drop in case we're redoing the values
    .drop(
        [c for c in [col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa] if c in df_labels_target_dynamic_raw.columns],
        axis=1
    )
    .merge(
        df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
        how='left',
        on=[col_new_cluster_val],
    )
)

df_labels_target_dynamic_raw.shape

39 <- clusters with 'Mature' in topic mix


,exclude_from_qa-count,exclude_from_qa-percent,exclude_from_qa-pct_cumulative_sum
keep,262,97.0%,97.0%
exclude from QA,8,3.0%,100.0%


(3517, 127)

In [ ]:
# df_labels_target_dynamic_raw.iloc[:5, -7:]

In [ ]:
mask_keep_clusters = (
    df_cluster_for_humans[col_exclude_from_qa] != val_exclude_from_qa_
)
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    .iloc[:7, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names,exclude_from_qa
1,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,Mature Themes and Adult Content,11,"t5_2wxtt9, t5_5cv619, t5_2wu5o, t5_2t240k, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1","jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",keep
2,0001-0001-0001-0001-0001-0002-0002-0002-0005-0009-0012-0014,k_1065_label,Mature Themes and Adult Content,13,"t5_wqh04, t5_to4e8, t5_2ba3ak, t5_42lxaq, t5_24jzkb, t5_2s9cd8, t5_24vux1, t5_4i3osf, t5_2hf2ft, t5_2z2x3, t5_4g3v2z, t5_4jmq4w, t5_rivb0","celeb_nylons, celebsintighttops, celebsinbusinesswear, celebmedia, strictlyhotties, emmerdalegirls, soapbabes, britishcelebritybabes, sexy_uk_girls, musicbabes, celebskart_pics, skysportnewsfemale, paddockwomen",keep
4,0001-0001-0002-0002-0002-0003-0003-0003-0006,k_0320_label,Mature Themes and Adult Content,10,"t5_4qepjs, t5_52oi1a, t5_59euk3, t5_31sgqc, t5_2al3kz, t5_4dgbcr, t5_2rd2l, t5_52xhgk, t5_4d1rbc, t5_51v2np","tadqeesyasin, ukhornythots, ukthotsx2, jensmith, joannachimonides, laetitiabrown, drama, sofiadeilamy, sofiaemilydeilamy, carlafrigoo2",keep
5,0001-0001-0002-0002-0002-0003-0003-0003-0006-0011-0014-0016-0025-0030,k_1840_label,Mature Themes and Adult Content,14,"t5_48i144, t5_2x72j, t5_4aukz3, t5_3ll0hi, t5_3ee9y6, t5_4u32j7, t5_72i29, t5_36nsk8, t5_3i2zbr, t5_35apq, t5_lg99y, t5_3hlch, t5_11rck8, t5_3jvo4","6kenza, chav, brogansmallfans, pritifit, mxelenaa, galacticawhore, carolinacutt, charlixcxsexy, davinamccall, grace_chatto, gracefituk, jorjasmith, jorjasmithpics, jadethirlwall",keep
6,0001-0001-0003-0003-0003-0004-0004-0004-0010,k_0320_label,Mature Themes and Adult Content | Celebrity,15,"t5_2s722, t5_2sk9j, t5_2t8y8, t5_2tvwu, t5_2rwx8, t5_31k2p, t5_2ylx0, t5_2u6ku, t5_2sdii, t5_2y16t, t5_5bxw5e, t5_4mudxz, t5_4at500, t5_30muol, t5_3dwus4","keiraknightley, karengillan, maisiewilliams, jennalouisecoleman, katebeckinsale, daisyridley, chloebennet, gilliananderson, kellybrook, georgiamayfoote, talialewiscolepics, rubyaeron, hollyrobinson, girlsinties, bowblouse",keep
7,0001-0001-0003-0003-0003-0004-0004-0004-0010-0018-0025-0028-0046,k_1560_label,Mature Themes and Adult Content | Celebrity,15,"t5_2ew45z, t5_q2cjh, t5_4tywry, t5_4qogng, t5_261zrr, t5_ohie3, t5_17nl3s, t5_4v254f, t5_5avkb7, t5_2yo3k, t5_2rz57, t5_2snpnh, t5_3fqet, t5_2sii9, t5_2ys6y","kadymcdermott, zaramcdermott, marybedford, mcloughlintwins, laurawoods, alicecassell, azzyland_pics, alexscott, rebeccaryanonly, hayleyorrantia, hayleywilliams, karengillanlegs, thehollywilloughby, bonniewright, laurenmayberry",keep
8,0001-0001-0003-0003-0003-0004-0004-0004-0010-0018-0025-0028-0046-0054-0061-0067-0089-0102-0113-0128-0137-0138,k_3927_label,Mature Themes and Adult Content | Celebrity,10,"t5_2zr9y, t5_2t9m5, t5_3dogba, t5_54xzk0, t5_2v71u, t5_2sxo1, t5_2vfo9, t5_2qx72, t5_owoic, t5_unq14","melissafumero, sarahmichellegellar, chanellemauricettee, charlottedyson, nataliealynlind, annakendrick, rachelriley, kristenbell, emmajones, emmamackey",keep


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Copy baseline cols for clean

In [ ]:
col_model_sort_order = 'model_sort_order'

# create new col as int for label so we can add a color scale when doing QA
col_new_cluster_val_int = 'cluster_label_int'
df_labels_target_dynamic_raw[col_new_cluster_val_int] = (
    df_labels_target_dynamic_raw[col_new_cluster_val]
    .str[-4:].astype(int)
)

# create new column for reddit link, makes QA easier
col_link_to_sub = 'link_to_sub'
link_prefix_ = 'www.reddit.com/r/'
df_labels_target_dynamic_raw[col_link_to_sub] = link_prefix_ + df_labels_target_dynamic_raw['subreddit_name']

# create lookup for rating col, might need to refresh it later in google sheets, though
col_rated_e = 'rated E'
# need to make it boolean for checkbox thing to work
df_labels_target_dynamic_raw[col_rated_e] = (
    df_labels_target_dynamic_raw['rating_short'] == 'E'
)

# Set expected col order
l_cols_clean_final_for_qa = [
    'subreddit_id',
    'subreddit_name',
    col_new_cluster_val_int,
    col_new_cluster_topic_mix,
    
    # insert inputs for QA cols
    'not country relevant',
    col_rated_e,
    'relevant to cluster/ other subreddits in cluster',
    'safe to show in relation to cluster',
    col_link_to_sub,

    # cols for notes
    'country relevance notes',
    'rating or cluster notest',
    #  # 'cluster relevance notes',

    col_subs_in_cluster_count,
    col_list_cluster_names,

    'allow_discovery',
    'users_l7',
    'total_users_in_subreddit_l28',
    'rating_name',

    'over_18',
    'rating_short',
    'primary_topic',
    col_subreddit_topic_mix,
    # why did a sub get marked as geo or culturally relevant?
    #  can use them to sort
    'geo_relevance_default',
    'relevance_percent_by_subreddit',
    'relevance_percent_by_country_standardized',
    'b_users_percent_by_subreddit',
    'e_users_percent_by_country_standardized',
    'd_users_percent_by_country_rank',
    col_model_sort_order,

    'posts_for_modeling_count',
    'c_users_percent_by_country',
    'users_in_subreddit_from_country_l28',
    'total_users_in_country_l28',
    
    col_new_cluster_val,
    col_new_cluster_name,
    col_new_cluster_prim_topic,
]

# copy existing columns from raw + 
l_cols_clean_existing = [c for c in l_cols_clean_final_for_qa if c in df_labels_target_dynamic_raw.columns]
l_cols_clean_new = [c for c in l_cols_clean_final_for_qa if c not in df_labels_target_dynamic_raw.columns]

# reorder cols in raw DF so it's easier to switch between the two tabs
df_labels_target_dynamic_raw = df_labels_target_dynamic_raw[
    reorder_array(l_cols_clean_existing, df_labels_target_dynamic_raw.columns)
]

# only copy subs that aren't in excluded clusters!!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_exclude_from_qa] != val_exclude_from_qa_]
    [l_cols_clean_existing]
    .copy()
)
# Add new columns and initialize them with empty strings
df_labels_target_dynamic_clean[l_cols_clean_new] = ''

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean,
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))

# re-order & rename the columns so their easier to see in google sheets
# Sorty by cluster label b/c sometimes a sub won't be clustered dynamically next to closest neighbors!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_clean[l_cols_clean_final_for_qa]
    .sort_values(by=[col_new_cluster_val, col_model_sort_order], ascending=True)
    # do final renaming only when saving, otherwise, it's a pain to adjust or look things up?
    .rename(columns={c: c.replace('_', ' ') for c in l_cols_clean_final_for_qa[5:]})
)

print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,262,0,13.06,12.00,20,


35 <- expected final col count
(3517, 130) <- df raw shape
(3421, 35) <- df clean shape


In [ ]:
df_labels_target_dynamic_clean.iloc[:5, :24]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,link to sub,country relevance notes,rating or cluster notest,subs in cluster count,list cluster subreddit names,allow discovery,users l7,total users in subreddit l28,rating name,over 18,rating short,primary topic,subreddit full topic mix,geo relevance default,relevance percent by subreddit,relevance percent by country standardized
4,t5_2wxtt9,jennifalconer,11,Mature Themes and Adult Content,,False,,,www.reddit.com/r/jennifalconer,,,11,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",None,115,424,None,f,None,Celebrity,Mature Themes and Adult Content | Celebrity,True,True,False
5,t5_5cv619,sarahjdunn,11,Mature Themes and Adult Content,,False,,,www.reddit.com/r/sarahjdunn,,,11,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",None,2281,5389,None,None,None,Celebrity,Mature Themes and Adult Content,True,True,True
6,t5_2wu5o,fiveheads,11,Mature Themes and Adult Content,,True,,,www.reddit.com/r/fiveheads,,,11,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",None,5124,20032,Everyone,f,E,Funny/Humor,Mature Themes and Adult Content,False,False,True
7,t5_2t240k,uk_babes_underrated,11,Mature Themes and Adult Content,,True,,,www.reddit.com/r/uk_babes_underrated,,,11,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",None,711,2678,Everyone,f,E,Celebrity,Mature Themes and Adult Content,False,True,True
8,t5_3cvnbw,uk_tv_girls,11,Mature Themes and Adult Content,,True,,,www.reddit.com/r/uk_tv_girls,,,11,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker",None,521,1843,Everyone,f,E,Celebrity,Mature Themes and Adult Content,True,True,True


## Create target to target list for FPR example

In [ ]:
%%time

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

  (46496, 9) <- df_ab.shape after removing matches to self
  (3421, 7) <- df_a_to_b.shape
CPU times: user 385 ms, sys: 2.36 ms, total: 388 ms
Wall time: 488 ms


In [ ]:
df_target_to_target_list.iloc[:5, :11]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count,list_cluster_subreddit_names,list_cluster_subreddit_ids
0,jennifalconer,t5_2wxtt9,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10,"sarahjdunn, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker","t5_5cv619, t5_2wu5o, t5_2t240k, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1"
1,sarahjdunn,t5_5cv619,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10,"jennifalconer, fiveheads, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker","t5_2wxtt9, t5_2wu5o, t5_2t240k, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1"
2,fiveheads,t5_2wu5o,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10,"jennifalconer, sarahjdunn, uk_babes_underrated, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker","t5_2wxtt9, t5_5cv619, t5_2t240k, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1"
3,uk_babes_underrated,t5_2t240k,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10,"jennifalconer, sarahjdunn, fiveheads, uk_tv_girls, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker","t5_2wxtt9, t5_5cv619, t5_2wu5o, t5_3cvnbw, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1"
4,uk_tv_girls,t5_3cvnbw,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10,"jennifalconer, sarahjdunn, fiveheads, uk_babes_underrated, eastenderswomen, rtgirls, neighboursbabes, hotchefs, jodiecomer, jodiewhittaker","t5_2wxtt9, t5_5cv619, t5_2wu5o, t5_2t240k, t5_3jhe9, t5_3g9c8, t5_2bfy1d, t5_5e16ui, t5_3k53x, t5_3mts1"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

In [ ]:
gspread.__version__

'5.1.1'

In [ ]:
# # %%time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'
country_name_sheet_ = 'GB'
target_abbrev_ = 'gb'
GSHEET_KEY = '1L_DuqMAYkf5pzipNcNLI_CvQkxJktMkKFu5pqliKhzg'
GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'


d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_raw': {
        'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    }
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)

Opening google worksheet: i18n GB subreddits and clusters - model v0.4.1 ...
Opening tab/sheet: subs_need_to_be_rated ...
Opening tab/sheet: raw_data_per_subreddit ...
Opening tab/sheet: raw_clusters_list_gb_gb ...
Opening tab/sheet: raw_clusters_fpr_gb_gb ...


## Save: Clean sheet to rate

In [ ]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'sub_raw', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_raw'])

In [ ]:
%%time
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

CPU times: user 115 ms, sys: 2.11 ms, total: 117 ms
Wall time: 3.69 s


{'spreadsheetId': '1L_DuqMAYkf5pzipNcNLI_CvQkxJktMkKFu5pqliKhzg',
 'updatedCells': 119770,
 'updatedColumns': 35,
 'updatedRange': 'subs_need_to_be_rated!A1:AI3422',
 'updatedRows': 3422}

## Save: df cluster for humans

In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

CPU times: user 8.29 ms, sys: 2.05 ms, total: 10.3 ms
Wall time: 317 ms


{'spreadsheetId': '1L_DuqMAYkf5pzipNcNLI_CvQkxJktMkKFu5pqliKhzg',
 'updatedCells': 1897,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_list_gb_gb!A1:G271',
 'updatedRows': 271}

## Save: target raw dynamic


In [ ]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [ ]:
%%time
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

CPU times: user 300 ms, sys: 18 ms, total: 318 ms
Wall time: 7.97 s


{'spreadsheetId': '1L_DuqMAYkf5pzipNcNLI_CvQkxJktMkKFu5pqliKhzg',
 'updatedCells': 299030,
 'updatedColumns': 85,
 'updatedRange': 'raw_data_per_subreddit!A1:CG3518',
 'updatedRows': 3518}

## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

In [ ]:
df_target_to_target_list.iloc[:5, :5]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count
0,jennifalconer,t5_2wxtt9,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10
1,sarahjdunn,t5_5cv619,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10
2,fiveheads,t5_2wu5o,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10
3,uk_babes_underrated,t5_2t240k,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10
4,uk_tv_girls,t5_3cvnbw,0001-0001-0001-0001-0001-0002-0002-0002-0004-0008-0011,k_0958_label,10


In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
    .update(
        [df_target_to_target_list.columns.values.tolist()] + 
        df_target_to_target_list.fillna('').values.tolist()
    )
)

CPU times: user 45.3 ms, sys: 3.02 ms, total: 48.3 ms
Wall time: 1.83 s


{'spreadsheetId': '1L_DuqMAYkf5pzipNcNLI_CvQkxJktMkKFu5pqliKhzg',
 'updatedCells': 23954,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_fpr_gb_gb!A1:G3422',
 'updatedRows': 3422}

### We can read the data back to confirm it's as expected

In [ ]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['sub_raw']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit_id,subreddit_name,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_names,primary_topic,rating_short,subreddit_full_topic_mix,rating_name,over_18,allow_discovery,geo_relevance_default,relevance_percent_by_subreddit,relevance_percent_by_country_standardized,b_users_percent_by_subreddit
0,t5_44faux,neighboursbabez,Mature Themes and Adult Content,6,"neighboursbabez, gilliananderson, kellybrook, azzyland_pics, alexscott, rebeccaryanonly",Mature Themes and Adult Content,,Mature Themes and Adult Content | Celebrity,,,,FALSE,TRUE,TRUE,0.209116
1,t5_2x56l,celebsinlingerie,Mature Themes and Adult Content,6,"celebsinlingerie, fitfootyplayers, zwiftracing, jennifalconer, sarahjdunn, fiveheads",Mature Themes and Adult Content,,Mature Themes and Adult Content | Celebrity,,,,FALSE,TRUE,TRUE,0.165381
2,t5_5ay13a,fitfootyplayers,Mature Themes and Adult Content,6,"celebsinlingerie, fitfootyplayers, zwiftracing, jennifalconer, sarahjdunn, fiveheads",,E,Mature Themes and Adult Content | Sports | Celebrity,Everyone,,,FALSE,TRUE,FALSE,0.288591
3,t5_3y6zfn,zwiftracing,Mature Themes and Adult Content,6,"celebsinlingerie, fitfootyplayers, zwiftracing, jennifalconer, sarahjdunn, fiveheads",Sports,E,Mature Themes and Adult Content | Sports,Everyone,,,FALSE,TRUE,FALSE,0.146104
4,t5_2wxtt9,jennifalconer,Mature Themes and Adult Content,6,"celebsinlingerie, fitfootyplayers, zwiftracing, jennifalconer, sarahjdunn, fiveheads",Celebrity,,Mature Themes and Adult Content | Celebrity,,f,,TRUE,TRUE,FALSE,0.431604


# Appendix


## Additional checks on cluster depth

In [ ]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

433


,count,percent,cumulative_percent
cluster_label,,,
0011-0018-0032-0043-0046-0059-0063-0088-0247-0494-0718-0801-1161-1365-1635-1745-2089-2317-2505-2721-2838-2879,50,1.4%,1.4%
0010-0016-0029-0039-0041-0052-0055-0076-0212-0422-0610-0684-0993-1166-1405-1494-1783-1985-2144-2323-2422-2456,40,1.1%,2.6%
0007-0011-0019-0026-0027-0036-0037-0047-0135-0273-0402-0452-0656-0774-0931-0990-1197-1329-1431-1558-1629-1654,32,0.9%,3.5%
0012-0019-0034-0050-0053-0067-0071-0099-0272-0548-0792-0881-1279-1510-1807-1925-2310-2561-2769-3004-3130-3176,23,0.7%,4.1%
0011-0018-0032-0043-0046-0059-0063-0088-0247-0494-0718-0801-1161-1365-1635-1745-2089-2317-2505-2720-2837-2878,22,0.6%,4.7%
0007-0011-0019-0026-0027-0036-0037-0047-0135-0272-0401-0451-0655-0773-0930-0989-1195-1327-1428-1555-1626-1651,19,0.5%,5.3%
0012-0020-0035-0052-0055-0069-0073-0102-0279-0564-0819-0911-1317-1558-1863-1987-2383-2642-2858-3096-3228-3277,19,0.5%,5.8%
0012-0019-0034-0049-0052-0065-0069-0097-0267-0538-0780-0868-1261-1489-1783-1900-2280-2528-2732-2965-3090-3136,18,0.5%,6.3%
0012-0019-0034-0050-0053-0067-0071-0099-0272-0548-0792-0881-1279-1510-1807-1925-2311-2563-2771-3006-3132-3178,17,0.5%,6.8%


count    433.000000
mean       8.120092
std        3.820557
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       50.000000
Name: count, dtype: float64

### How deep are the clusters?

Looks like some peaks around 100, 300, 1k, and 4k clusters.

In [ ]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
)

21


,count,percent,cumulative_percent
cluster_label_k,,,
k_0013_label,33,0.9%,0.9%
k_0023_label,28,0.8%,1.7%
k_0041_label,27,0.8%,2.5%
k_0059_label,15,0.4%,2.9%
k_0063_label,22,0.6%,3.6%
k_0085_label,47,1.3%,4.9%
k_0118_label,349,9.9%,14.8%
k_0320_label,435,12.4%,27.2%
k_0657_label,328,9.3%,36.5%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )

# Filter out subs [extra]

UPDATE: For now let's include all the subreddits for QA because this list could potentially help us rate/flag subreddits that aren't rated or mis-rated and have a lot of traffic.

Now that we have even more clusters (over 3,000), it's harder to figure out where to set the threshold for clusters to exclude.

--- 

The main use case for now are SFW subs, we could save some QA time by excluding these subs:
- Exclude NSFW clusters
- Exclude place subs


~We'll use the cluster labels to discard subreddits because~
- many of the DE subreddits don't have a `primary_topic`
- if the majority of subs for a subreddits are NSFW, then we wouldn't want to recommend those anyway

In [ ]:
# # we can see that the subreddit count changes as we go 
# #  from shallow to deeper cluster counts
# value_counts_and_pcts(
#     df_labels_target['k_0118_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3145_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3927_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# # And the count is slightly different from the known primary topics
# #  We still have a large number of subs w/o a primary topic
# value_counts_and_pcts(
#     df_labels_target['primary_topic'],
#     count_type='subreddits',
#     reset_index=True,
#     add_col_prefix=False,
# )

In [ ]:
print(f"{df_labels_target.shape} <- Shape before filtering")

l_manual_subs_to_remove = [
    'sexmeets1', 'fuck',
]
col_cluster_filter = 'k_3145_majority_primary_topic'
df_labels_target_clean = (
    df_labels_target[df_labels_target[col_cluster_filter] != 'Mature Themes and Adult Content']
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping NSFW clusters")

l_sensitive_topics = [
    'Military', 'Gender', 'Addiction Support',
    'Medical and Mental Health', 'Sexual Orientation',
    'Culture, Race, and Ethnicity',
]
df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean[col_cluster_filter].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Sensitive clusters")

df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['primary_topic'].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping SENSITIVE subreddits")


df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['subreddit_name'].isin(l_manual_subs_to_remove)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Manual list of subreddits")

print(f"  ** TODO: instead of excluding place subs, add logic to map hierarchy **")
# df_labels_target_clean = (
#     df_labels_target_clean[df_labels_target_clean['primary_topic'] != 'Place']
# )
# print(f"{df_labels_target_clean.shape} <- Shape after dropping Place subreddits")

(3516, 74) <- Shape before filtering
(3272, 74) <- Shape after dropping NSFW clusters
(3063, 74) <- Shape after dropping Sensitive clusters
(3032, 74) <- Shape after dropping SENSITIVE subreddits
(3032, 74) <- Shape after dropping Manual list of subreddits
  ** TODO: instead of excluding place subs, add logic to map hierarchy **
